In [1]:
from PIL import Image
import pytesseract as tess


In [2]:
im=Image.open("test_image\\test1.jpeg")

In [3]:
print(im)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x771 at 0x1EDA09878D0>


In [4]:
image_data=tess.image_to_string(im)
print(image_data)

Independent
4¢ | University,
s° Bangladesh

Sourav Kumar Saha
Student
1D: 1930188

dK

wut LAL



#using open ai vision model to get the text form the image


In [5]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import base64
load_dotenv()

True

In [6]:
# OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
client=OpenAI()

In [7]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
image_path = "test_image\\test1.jpeg"
base64_image = encode_image(image_path)

In [8]:
responce=client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[ 
        {
            "role":"system",
            "content":"You are a helpful assistant."
        },
        {
            "role":"user",
            "content":[
                {
                    "type":"text",
                    "text":"""extract text from image of id card. Text like university name, student name,
                    student id. give me the extracted information as dict format key value piar.
                    example:
                    [
                        {
                            University: university name,
                            Student Name: student name, 
                            ID: student id,
                        },
                        {
                            University: university name,
                            Student Name: student name, 
                            ID: student id,
                        },
                    
                    ]
                    
                    if the  there is multiple id there stracture data in list of dictionary format as given .
                    and just give me the extracted information in text format. no extra information.
                    """
                },
                {
                    "type":"image_url",
                    "image_url":{"url": f"data:image/jpeg;base64,{base64_image}"}
                }
            ]
        }
    ],
    max_tokens=300,
)

In [9]:
print(responce)


ChatCompletion(id='chatcmpl-Aifyq7rvae2DGESQkdttdCx3ANOLN', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n[\n    {\n        "University": "Independent University, Bangladesh",\n        "Student Name": "Sourav Kumar Saha",\n        "ID": "1930188"\n    }\n]\n```', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735211168, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_d02d531b47', usage=CompletionUsage(completion_tokens=42, prompt_tokens=14312, total_tokens=14354, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [10]:
responce.choices[0].message.content


'```json\n[\n    {\n        "University": "Independent University, Bangladesh",\n        "Student Name": "Sourav Kumar Saha",\n        "ID": "1930188"\n    }\n]\n```'

In [11]:
responce_data=responce.choices[0].message.content
responce_data

'```json\n[\n    {\n        "University": "Independent University, Bangladesh",\n        "Student Name": "Sourav Kumar Saha",\n        "ID": "1930188"\n    }\n]\n```'

In [43]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_openai import ChatOpenAI
import json

In [45]:
llm =ChatOpenAI(temperature=0,model="gpt-4o-mini")

In [70]:
get_info_prompt = PromptTemplate(
    input_variables=["extracted_data"],
    template="""
    You are an AI assistant that performs calculations and transformations on the given input data.

    Input data: {extracted_data}
    Perform the following tasks on the input data and return the results in a list of dictionaries:

    Task 1: Convert all text to uppercase.
    Task 2: Create a short name for the university name. 
        Example: "north south university" -> "NSU", 
                 "american international university bangladesh" -> "AIUB".
    Task 3: Create a short name for the student name. 
        Example: "avijit kumar saha" -> "AKS", 
                 "isshan saha" -> "IS".
    Task 4: Extract the year of admission and semester name from the student ID.
        - Year of admission: The first two digits of the ID represent the starting year. Example: if ID number is kind of this "2010188" -> "2020".
        - Semester name: The third character or number of the ID indicates the semester (1 -> Autumn, 2 -> Spring, 3 -> Summer). example: if ID number is kind of this "2010188" -> "Autumn".("2010188" example id)
    Task 5: Calculate the estimated graduation year. 
        - Graduation is assumed to take 4 years. 
        - Example: If a student starts in "2019 Spring", they graduate in "2023 Spring".

    Return the output in the following format:
    [
        dict(
            "university": "UNIVERSITY NAME",
            "short_university_name": "SU",
            "student_id": "STUDENT ID",
            "student_name": "STUDENT NAME",
            "short_student_name": "SN",
            "year_of_admission": "YEAR",
            "semester": "SEMESTER",
            "estimated_graduation_year": "YEAR",
        )
    ]

    Provide only the extracted information in the given format. No extra information.
    """
)


In [71]:
proccess_data_chain=LLMChain(llm=llm,prompt=get_info_prompt)

In [72]:
# data_as_string = json.dumps(responce_data, indent=2)
result = proccess_data_chain.run(extracted_data=responce_data)


In [73]:
result

'```json\n[\n    {\n        "university": "INDEPENDENT UNIVERSITY, BANGLADESH",\n        "short_university_name": "IUB",\n        "student_id": "1930188",\n        "student_name": "SOURAV KUMAR SAHA",\n        "short_student_name": "SKS",\n        "year_of_admission": "2019",\n        "semester": "Autumn",\n        "estimated_graduation_year": "2023"\n    }\n]\n```'

In [76]:
json_data=result.replace("```json","").replace("```","").strip()
data=json.loads(json_data)
data

[{'university': 'INDEPENDENT UNIVERSITY, BANGLADESH',
  'short_university_name': 'IUB',
  'student_id': '1930188',
  'student_name': 'SOURAV KUMAR SAHA',
  'short_student_name': 'SKS',
  'year_of_admission': '2019',
  'semester': 'Autumn',
  'estimated_graduation_year': '2023'}]

In [22]:
# ectract_image_data_prompt=PromptTemplate(
#     input_variables=["image_base64"],
    
#     template="""
#         you are an ai assistant. you have to extract text from image of id card. Text like university name, student name,
#         student id. universty establis year. give the extracted impormation in list of dictionary format.
#         example:
#         [
#            {{
#                 University: university name,
#                 Student Name: student name, 
#                 ID: student id,
#             }},
#             {{
#                 University: university name,
#                 Student Name: student name, 
#                 ID: student id,
#             }},
                    
#         ]
                    
#         if the  there is multiple id there stracture data in list of dictionary format as given .
        
    
    
#     """  
# )

# extract_image_data_chain=LLMChain(llm=llm,prompt=ectract_image_data_prompt)


In [17]:
# extract_image_data_chain.invoke({"image_base64":base64_image})